In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchvision import datasets, transforms
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import sys
import copy
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

# 하이퍼파라미터 설정
batch_size = 64
epochs = 10
lr = 0.01
PCA_dim = 8
CLS_num = 2


# 데이터 전처리 설정
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# 데이터셋 로드
data = load_breast_cancer()
X = data.data
y = data.target

X, y = make_classification(n_samples=1000, n_features=8, n_informative=5, n_redundant=3, n_classes=2, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

"""
train_dataset = datasets.MNIST('../data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('../data', train=False, transform=transform)
train_mask = (train_dataset.targets >= 0) & (train_dataset.targets <= CLS_num-1)
test_mask = (test_dataset.targets >= 0) & (test_dataset.targets <= CLS_num-1)

train_dataset.data = train_dataset.data[train_mask]
train_dataset.targets = train_dataset.targets[train_mask]
test_dataset.data = test_dataset.data[test_mask]
test_dataset.targets = test_dataset.targets[test_mask]

# 데이터를 NumPy 배열로 변환
x_train = train_dataset.data.numpy().reshape(-1, 784)
x_test = test_dataset.data.numpy().reshape(-1, 784)
y_train = train_dataset.targets.numpy()
y_test = test_dataset.targets.numpy()"""

# PCA로 feature 축소
pca = PCA(n_components=PCA_dim)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)

# 정규화 (표준 스케일러 사용)
scaler = StandardScaler()
x_train_pca = scaler.fit_transform(x_train_pca)
x_test_pca = scaler.transform(x_test_pca)

# PyTorch Tensor로 변환
x_train_pca, y_train = torch.tensor(x_train_pca, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long)
x_test_pca, y_test = torch.tensor(x_test_pca, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long)


class Feature_data_loader(Dataset):
    def __init__(self,x_train,y_train):
        self.feature1 = x_train
        temp = copy.deepcopy(x_train)
        shuffle = torch.randperm(len(temp))
        self.feature2 = temp[shuffle]

        self.y1 = y_train
        temp_y = copy.deepcopy(y_train)
        self.y2 = temp_y[shuffle]

    def __len__(self):
        return len(self.feature1)
    def __getitem__(self,idx):
        input1 = self.feature1[idx]
        input2 = self.feature2[idx]
        if self.y1[idx] == self.y2[idx]:
            label = torch.tensor(1.).float()
        else:
            label = torch.tensor(0.).float()
        return [input1,input2],label


# DataLoader 생성


feature_loader = DataLoader(Feature_data_loader(x_train_pca, y_train.float()),batch_size=batch_size,shuffle=True)
test_feature_loader = DataLoader(Feature_data_loader(x_test_pca, y_test.float()),batch_size=batch_size,shuffle=False)
train_loader = DataLoader(TensorDataset(x_train_pca, y_train), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(x_test_pca, y_test), batch_size=batch_size, shuffle=False)


In [29]:
def accuracy(pred, true):
    # 예측값이 로짓 혹은 확률값인 경우, 최대 값을 가진 인덱스를 구함 (가장 확률이 높은 클래스)
    pred = pred.detach().cpu()
    true = true.cpu()
    try:
        pred_labels = torch.argmax(pred, dim=1)
    except:
        pred_labels = torch.round(pred)
    # 예측 레이블과 실제 레이블이 일치하는 경우를 계산
    correct = (pred_labels == true).sum()
    # 정확도를 계산
    acc = correct / true.size(0)
    return acc.item()

class Early_stop_train():
    def __init__(self,model, optimizer, criterion):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion



        self.loss_list = [1e100]
        self.stop_count = 0

    def train_model(self,train_loader,test_loader=None ,epochs=200,res = 10):
        self.model.train()
        for epoch in range(epochs):
            if self.stop_count>=res:
                break
            loss_val,_ = self.test(test_loader)
            self.loss_list.append(loss_val)

            if self.loss_list[-1]>=np.min(self.loss_list[:-1]):
                self.stop_count+=1
            else:
                self.stop_count = 0
            loss_list = []
            acc_list = []
            for X_train,y_train in train_loader:

                self.optimizer.zero_grad()
                output = self.model(X_train)

                loss = self.criterion(output.squeeze(), y_train)

                loss.backward()
                self.optimizer.step()
                loss_list.append(loss.item())
                acc = accuracy(output,y_train)
                acc_list.append(acc)

                sys.stdout.write(f"\rEpoch {epoch+1} Loss {np.mean(loss_list):4f} acc : {np.mean(acc_list):4f} stop count : {self.stop_count}")


    def test(self,test_loader):
        if test_loader is None:
            return 0,0
        else:
            #self.model.eval()
            test_loss = 0
            correct = 0
            with torch.no_grad():
                for data, target in test_loader:
                    data, target = data, target
                    output = self.model(data)
                    test_loss += self.criterion(output.squeeze(), target).item()

                    correct += accuracy(output,target)*len(output)

            print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)')
            return test_loss,correct

In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pennylane as qml
from pennylane import numpy as np
from collections import OrderedDict
import math

result_list_classical = []

# 데이터 로드 및 전처리
"""
data = load_breast_cancer()
X = data.data
y = data.target

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=PCA_dim)
X_pca = pca.fit_transform(X_scaled)
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.PCA_dim, random_state=seed)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)"""

# Pennylane 장치 설정
dev = qml.device("default.qubit", wires=PCA_dim)


def ZZFeatureMapLayer(features, wires):
    """사용자 정의 ZZFeatureMap 레이어"""
    index = 0
    for i in wires:
        qml.Hadamard(wires=i)
        qml.RZ(features[:,index], wires=i)
        index += 1

    for j in range(0, len(wires)-1):
        qml.CNOT(wires=[j, j+1])
        qml.RZ((features[:,index]), wires=j+1)
        qml.CNOT(wires=[j, j+1])
        index+=1

def ansatz(params):
    for j in range(len(params)):
        # 각 큐비트에 대해 RX, RY, RZ 회전 적용
        for i in range(len(params[0])):
            qml.RY(params[j, i, 0], wires=i)
            qml.RZ(params[j, i, 1], wires=i)

        # 인접한 큐비트 간 CNOT 게이트로 엔탱글링
        if j == len(params)-1:
            pass
        else:
            for i in range(len(params[0])-1):
                qml.CNOT(wires=[i, i+1])


# 양자 레이어 정의
@qml.qnode(dev, interface='torch', diff_method="backprop")
def QuantumLayer(features,params):
    ZZFeatureMapLayer(features, wires=range(PCA_dim))
    ansatz(params)
    return qml.probs(wires=range(math.ceil(math.log2(CLS_num))))


## 양자 커널
@qml.qnode(dev, interface='torch', diff_method="backprop")
def Kernal(features1,features2):
    ZZFeatureMapLayer(features1, wires=range(PCA_dim))
    qml.adjoint(ZZFeatureMapLayer)(features2,wires=range(PCA_dim))
    return qml.probs(wires=range(PCA_dim))


class Feature_model(nn.Module):
    def __init__(self):
        super(Feature_model,self).__init__()
        self.cls = nn.Sequential(OrderedDict([('cls1', nn.Linear(PCA_dim,PCA_dim*4)),('relu1', nn.ReLU()),('cls2', nn.Linear(PCA_dim*4,PCA_dim*4)),('relu1', nn.ReLU()),('cls3', nn.Linear(PCA_dim*4,PCA_dim*2-1)),('sigmoid', nn.ReLU())]))
        self.Kernal = Kernal
    def forward(self,inputs):
        epsilon = 1e-6
        input1 = inputs[0]
        input2 = inputs[1]
        input1 = self.cls(input1)*np.pi
        input2 = self.cls(input2)*np.pi
        output = self.Kernal(input1,input2)
        output = output.type(torch.float32)
        return output[:,0].clamp(min=epsilon, max=1-epsilon)





# 하이브리드 모델 정의
class HybridModel(nn.Module):
    def __init__(self):
        super(HybridModel, self).__init__()
        self.cls = feature_model.cls

        self.quantum_layer = QuantumLayer
        self.Q_params = nn.Parameter((torch.rand([PCA_dim,PCA_dim,2])*2-1)*np.pi,requires_grad=True)
    def forward(self, x):
        x = self.cls(x)*np.pi
        #print(qml.draw(self.quantum_layer)(x,self.Q_params))
        quantum_output = self.quantum_layer(x,self.Q_params)
        quantum_output = quantum_output.type(torch.float32)
        return torch.log(quantum_output)

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.cls_layer_1 = nn.Linear(PCA_dim,PCA_dim*PCA_dim)
        self.cls_layer_2 = nn.Linear(PCA_dim*PCA_dim,PCA_dim*PCA_dim-1)
        self.output_layer = nn.Linear(PCA_dim*PCA_dim-1,PCA_dim)
    def forward(self, x):
        x = self.cls_layer_1(x)
        x = nn.ReLU()(x)
        x = self.cls_layer_2(x)
        x = nn.ReLU()(x)
        output = self.output_layer(x)
        return output
# 모델, 손실 함수, 최적화 설정


feature_model = Feature_model(); criterion = nn.BCELoss()
#model = Model(); criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(feature_model.parameters(), lr=0.001)


# 모델 학습 및 평가
train_process = Early_stop_train(feature_model, optimizer, criterion)
train_process.train_model(feature_loader,test_feature_loader,epochs=50,res=15)
model = HybridModel(); criterion = nn.NLLLoss()
for param in model.cls.parameters():
    param.requires_grad = False
#model.load_state_dict(para_dict)
optimizer = optim.Adam(model.parameters(), lr=0.01)
print('\n\n Test start \n\n')
train_process = Early_stop_train(model, optimizer, criterion)
train_process.train_model(train_loader,test_loader,epochs=50,res=5)

_,acc = train_process.test(test_loader)
result_list_classical.append(acc)
print(f"Test Accuracy: {acc:.2f}")




Test set: Average loss: 3.0220, Accuracy: 102.0/200 (51%)
Epoch 1 Loss 0.858949 acc : 0.501202 stop count : 0
Test set: Average loss: 3.5289, Accuracy: 115.0/200 (58%)
Epoch 2 Loss 0.740482 acc : 0.568510 stop count : 1
Test set: Average loss: 3.6243, Accuracy: 108.0/200 (54%)
Epoch 3 Loss 0.688772 acc : 0.603365 stop count : 2
Test set: Average loss: 3.3827, Accuracy: 111.0/200 (56%)
Epoch 4 Loss 0.656782 acc : 0.632212 stop count : 3
Test set: Average loss: 3.1398, Accuracy: 109.0/200 (54%)
Epoch 5 Loss 0.625734 acc : 0.659856 stop count : 4
Test set: Average loss: 3.1329, Accuracy: 113.0/200 (56%)
Epoch 6 Loss 0.601383 acc : 0.686298 stop count : 5
Test set: Average loss: 3.0653, Accuracy: 116.0/200 (58%)
Epoch 7 Loss 0.575819 acc : 0.715144 stop count : 6
Test set: Average loss: 2.9768, Accuracy: 117.0/200 (58%)
Epoch 8 Loss 0.567589 acc : 0.717548 stop count : 0
Test set: Average loss: 2.9588, Accuracy: 118.0/200 (59%)
Epoch 9 Loss 0.547498 acc : 0.735577 stop count : 0
Test set:

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 데이터 생성
X, y = make_classification(n_samples=1000, n_features=8, n_informative=5, n_redundant=3, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# PCA로 특성 수 줄이기 (옵션)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

pca = PCA(n_components=8)  # 차원 축소
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# 텐서로 변환
X_train_tensor = torch.tensor(X_train_pca, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_pca, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# 신경망 모델 정의
class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(8, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

model = BinaryClassifier()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 훈련
def train_model(model, criterion, optimizer, epochs=200):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train_tensor).squeeze()
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()

train_model(model, criterion, optimizer)

# 평가
def evaluate_model(model, X, y):
    model.eval()
    with torch.no_grad():
        outputs = model(X).squeeze()
        predicted = (outputs > 0.5).float()
        accuracy = (predicted == y).float().mean()
    return accuracy

accuracy = evaluate_model(model, X_test_tensor, y_test_tensor)
accuracy

tensor(0.8850)